In [1]:
import sys
sys.path.append('/Users/spindicate/Documents/programming/zooscraper/')

In [2]:
import django
django.setup()

In [3]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [5]:
import numpy as np
import pandas as pd
import xarray as xr

from casestudy.models import Country, Region, City, Pollutant

In [6]:
# MUST ASSIGN CITIES TO REGIONS!
# REDUCE POPULATION DATA TO REGIONS IN DATABASE
# LOOK FOR CITIES IN AQ IN REDUCED POP DATA

In [8]:
aq_path = '/Users/spindicate/Documents/docs/covid19/pollution/waqi-covid19-airqualitydata-2020.csv'
df_aq = pd.read_csv(aq_path, delimiter=',', engine='python', comment='#', error_bad_lines=False)

In [10]:
path = '/Users/spindicate/Documents/docs/covid19/pops/gpw-v4-admin-unit-center-points-population-estimates-rev11_global_csv/gpw_v4_admin_unit_center_points_population_estimates_rev11_global.csv'
df_exUS = pd.read_csv(path)

In [327]:
# path = '/Users/spindicate/Documents/docs/covid19/pops/gpw-v4-admin-unit-center-points-population-estimates-rev11_global_csv/'
# files = [file for file in os.listdir(path) if 'usa' in file]
# dfs = [pd.read_csv(path + file) for file in files]
# df_us = pd.concat(dfs)

In [328]:
# ds_exUS = xr.Dataset.from_dataframe(df_exUS)
# # Need to Amend Chinese names
# for i in range(4):
#     name_col = 'NAME' + str(i + 1)
#     ds_exUS[name_col] = ds_exUS[name_col].fillna('')
#     ds_exUS[name_col] = ds_exUS[name_col].str.title()

# df_exUS = xr.Dataset.to_dataframe(ds_exUS)

In [11]:
# regions = [name for region in Region.objects.values_list('name', 'name_alt') for name in region if name]
# df_us = df_us[(df_us['COUNTRYNM'].isin(regions)) | (df_us['NAME1'].isin(regions)) ]
# countries_used = Region.objects.filter(country_key__isnull=False).values_list('country_key__alpha2', flat=True)
# df_aq = df_aq[df_aq.Country.isin(countries_used)]
df_aq = df_aq[df_aq['Country'] == 'BR']
# df_aq['City'] = df_aq['City'].str.title()

In [15]:
df_aq.City.unique()

array(['São José dos Campos', 'Vitória', 'São Paulo'], dtype=object)

In [12]:
# br_cities = list(df_aq.City.unique()).copy()

In [13]:
br_cities

['São José dos Campos', 'Vitória', 'São Paulo']

In [667]:
City.objects.filter(region__country_key__alpha2='US').values()

<QuerySet [{'id': 518, 'name': 'Fort Worth', 'region_id': 67}, {'id': 519, 'name': 'Tallahassee', 'region_id': 64}, {'id': 520, 'name': 'Queens', 'region_id': 75}, {'id': 521, 'name': 'Tucson', 'region_id': 80}, {'id': 522, 'name': 'Nashville', 'region_id': 96}, {'id': 523, 'name': 'Madison', 'region_id': 93}, {'id': 524, 'name': 'Salem', 'region_id': 77}, {'id': 525, 'name': 'San Francisco', 'region_id': 76}, {'id': 526, 'name': 'Albuquerque', 'region_id': 56}, {'id': 527, 'name': 'Salt Lake City', 'region_id': 95}, {'id': 528, 'name': 'Denver', 'region_id': 63}, {'id': 529, 'name': 'Saint Paul', 'region_id': 82}, {'id': 530, 'name': 'Columbus', 'region_id': 92}, {'id': 531, 'name': 'San Diego', 'region_id': 76}, {'id': 532, 'name': 'Philadelphia', 'region_id': 69}, {'id': 533, 'name': 'San Antonio', 'region_id': 67}, {'id': 534, 'name': 'Las Vegas', 'region_id': 86}, {'id': 535, 'name': 'Jacksonville', 'region_id': 64}, {'id': 536, 'name': 'Miami', 'region_id': 64}, {'id': 537, 'name

In [18]:
region = Region.objects.get(name='Sao Paulo')

In [662]:
# reg = Region.objects.get(name='Oklahoma', country_key__alpha2='US')

In [19]:
City.objects.create(name='São Paulo', region=region)

<City: City object (581)>

In [13]:
# Loop thru population; find city name
def find_city(city, df_exUS):
    df_city = df_exUS[
        (df_exUS['NAME1'] == city) | (df_exUS['NAME2'] == city) | (df_exUS['NAME3'] == city) | (df_exUS['NAME4'] == city)
        | (df_exUS['NAME3'] == city) | (df_exUS['NAME4'] == city)
        | (df_exUS['NAME1'].str.contains(city)) | (df_exUS['NAME2'].str.contains(city)) | (df_exUS['NAME3'].str.contains(city)) 
        | (df_exUS['NAME4'].str.contains(city))
        | (df_exUS['NAME3'].str.contains(city)) | (df_exUS['NAME4'].str.contains(city))
    ]

    potential_regions = []
    if df_city.empty:
        print ('there is no city in pop', city)
    for col in ['NAME1', 'NAME2']:
        potential_regions += df_city[col].unique().tolist()
        
    return potential_regions

In [333]:
cities_with_multiple_regions = []
cities_with_no_regions = []
country_with_multiple_regions = []
with transaction.atomic():
    for country, df_country in df_aq.groupby('Country'):
        for city, df_g in df_country.groupby('City'):
            country_obj = Country.objects.get(alpha2=country)
            try:
                region = Region.objects.get(Q(name__in=[country_obj.name, country_obj.alt1]) | Q(name_alt__in=[country_obj.name, country_obj.alt1]))
                city_obj = City(name=city, region=region)
#                 city_obj.save()
            except Region.MultipleObjectsReturned:
                try:
                    country = Country.objects.get(alpha2=country)
                    region = Region.objects.get(country_key=country)
                except: 
                    country_with_multiple_regions.append((city, country))
            except Region.DoesNotExist:
                try:
                    City.objects.get(name=city, region__country_key__alpha2=country)
                except:
                    potential_regions = set(find_city(city, df_us))
                    reg_objs = Region.objects.filter(Q(name__in=potential_regions) | Q(name_alt__in=potential_regions)).filter(country_key=country_obj)
                    if reg_objs.count() == 1:
                        pass
                    elif reg_objs.count() == 0:
                        cities_with_no_regions.append((city, country, potential_regions, reg_objs))
                    else:
                        cities_with_multiple_regions.append((city, country, potential_regions, reg_objs))

there is no city in pop Albuquerque
there is no city in pop Atlanta
there is no city in pop Boston
there is no city in pop Brooklyn
there is no city in pop Chicago
there is no city in pop Detroit


KeyboardInterrupt: 

In [92]:
# UPDATE REGION FOR COUNTRY FOREIGN KEY

country_names = list(Region.objects.values_list('country', flat=True).distinct())

countries = Country.objects.values_list('name', 'alt1', 'alpha2', 'alpha3')

countries = [name for country in countries for name in country]

missing_names = [country_name for country_name in country_names if country_name not in countries]

missing_names

from django.db.models import Q
for region in Region.objects.all():
    try:
        country = Country.objects.get(
            Q(name__in=[region.country, region.country_alt]) | Q(alpha3__in=[region.country, region.country_alt])
            | Q(alt1__in=[region.country, region.country_alt])
        )
        region.country_key=country
        region.save()
    except:
        print (region.name)

In [7]:
df_ital = df_aq[df_aq['Country'] == 'IT']

In [10]:
Pollutant.objects.filter(city__region__country_key__name='Italy').delete()

(6606, {'casestudy.Pollutant': 6606})

In [13]:
pollus_objs = []
for i, row in df_ital.iterrows():
    city = City.objects.get(name=row['City'])
    pollus_objs.append(
        Pollutant(
            date=row['Date'], city=city, pollutant=row['Specie'],
            count=row['count'], minimum=row['min'], maximum=row['max'],
            median=row['median'], variance=row['variance']
        )
    )


In [14]:
max_bulk_create(pollus_objs)

NameError: name 'max_bulk_create' is not defined